In [ ]:
# import the necessary packages
from imutils import paths 
import argparse
import imutils
import json 
import cv2
import os
import pandas as pd
import numpy as np

# tools for parallelism
from multiprocessing import Pool
from functools import partial

# separate function to compute mask for one landmark
from utils.utils import one_key_mask

In [ ]:
# original image size and intended image size
H = 348
W = 464

h = 256
w = 256

# number of facial landmarks
KEYPOINTS = 54

In [ ]:
def generate_masks(jsonFolder):
    
    # folder where generated masks will be stored
    masksFolder = jsonFolder.replace('/json', '/masks')
    if not os.path.exists(masksFolder):
        os.makedirs(masksFolder)

    # paths to json files
    jsonPaths = list(paths.list_files(jsonFolder, validExts="json"))
    jsonPaths = sorted(jsonPaths)

    # loop over the json files
    for ind, jsonPath in enumerate(jsonPaths, 1):

        print("[INFO] Processing {} file ({}/{})".format(jsonPath.split("/")[-1], ind, len(jsonPaths)))

        # opening the json file 
        f = open(jsonPath,) 

        # returns the json object as a dictionary 
        data = json.load(f) 

        # extracting filename
        filename = jsonPath.split("/")[-1]
        filename = filename.split(".json")[0]

        # extracting trial data from filename
        sub_id, trial_id, exp_id, pos_id, *rest = filename.split("_")
        info = "{}_{}_{}".format(sub_id, trial_id, pos_id)

        # if image is challenging, skip it
        df = pd.read_csv(os.path.join(datasetPath, "challenging_images.csv"))

        if ((df['sub_id'] == int(sub_id)) & (df['trial_id'] == int(trial_id)) & (df['pos_id'] == int(pos_id))).any():
            print('Excluding: ', info)
            continue

        # iterating through the shapes
        landmarks = []
        for shape in data['shapes']:
            # face label corresponds to bounding boxes
            if shape['label'] == 'face':
                [[xs, ys], [xe, ye]] = shape['points']
                (xs, ys, xe, ye) = (int(xs), int(ys), int(xe), int(ye))

            # other labels correspond to landmarks
            elif shape['label'] != 'face':
                # iterating through the landmarks
                for point in shape['points']:
                    (x, y) = point
                    (x, y) = (int(x), int(y))

                    # adjust the landmark positions w.r.t. bounding boxes and new image size
                    (crop_x, crop_y) = (x-xs, y-ys)
                    (crop_x, crop_y) = (crop_x / (xe-xs) * w, crop_y / (ye-ys) * h)
                    landmarks.append([crop_y, crop_x])

        landmarks = np.array(landmarks)

        # array to contain the masks for each point
        B = np.zeros((KEYPOINTS, h, w))

        # mask generation is computed on CPU
        # hence, the landmark computation is manually divided to 6 processors
        # separate landmarks to 6 subsets for parallel computation
        keys1 = (list(range(0, 9)))
        keys2 = (list(range(9, 18)))
        keys3 = (list(range(18, 27)))
        keys4 = (list(range(27, 36)))
        keys5 = (list(range(36, 45)))
        keys6 = (list(range(45, 54)))

        with Pool(processes=6) as pool:
            b1 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys1)
            b2 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys2)
            b3 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys3)
            b4 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys4)
            b5 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys5)
            b6 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys6)

            B[0:9, :, :] = np.array(b1)
            B[9:18, :, :] = np.array(b2)
            B[18:27, :, :] = np.array(b3)
            B[27:36, :, :] = np.array(b4)
            B[36:45, :, :] = np.array(b5)
            B[45:54, :, :] = np.array(b6)

        # combining to one mask
        mask = np.sum(B, axis=0)
        mask = np.clip(mask, 0, 1)
        mask = mask * 255.0
        mask = mask.astype('uint8')

        # saving the mask
        maskPath = os.path.join(masksFolder, "{}.png".format(filename))
        cv2.imwrite(maskPath, mask)

        # Closing file 
        f.close()

In [ ]:
generate_masks('dataset/gray/train/json')
generate_masks('dataset/gray/val/json')
generate_masks('dataset/gray/test/json')